In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import yfinance as yf
import scipy.optimize as opt
import scipy.stats as st
from scipy.special import gamma

In [2]:
spy = yf.download('SPY', start='2013-01-01', end='2022-12-01')['Adj Close']

[*********************100%***********************]  1 of 1 completed


## 1 a scratching parameter for GARCH (1,1)

In [3]:
spy_pct = spy.pct_change()
# detrend the return series
at = spy_pct-np.mean(spy_pct)
at = np.array(at[1:])
# need to drop na term

In [4]:
def sig_garch11(a0,a1,b,at):
    # at from a0 to at , length t+1
    n = len(at)
    sig = np.zeros(n)
    for i in range(len(at)):
        if i == 0 :
            sig[i] = a0/(1-a1-b)
        else:
            sig[i] = a0 + a1*at[i-1]**2 + b* sig[i-1]
    return sig
# the last sig doesnt match any output

In [5]:
def ll_garch11(para, at, distr='N'):
    a0 = para[0]
    a1= para[1]
    b = para[2]

    sig = sig_garch11(a0,a1,b,at)
    n = len(at)
    L = 0
    if distr == 'N':
        L += (-n/2)*np.log(2*np.pi)
        L += np.sum(-np.log(sig)-at**2/sig)/2
    if distr == 'GED':
        #need to adjust v (the last additional parameter), so 1/v is integer and we can use factorial
        v = para[-1]
        iv = 1/v
        lbd = np.sqrt((2**(-2*iv)*gamma(iv)/gamma(3*iv)))
        L += n*(np.log(v)-np.log(lbd)-(1+iv)*np.log(2)-np.log(gamma(iv)))
        L -= sum(np.log(sig))/2
        L -= 1/2* sum((at**2/(sig* (lbd**2)))**(v/2)) 
    return -L

In [6]:
w0 = [0.1,0.05,0.92]
bnds = ((0.001,None), (0.001,None),(0.001,None))
w_n = opt.minimize(ll_garch11, w0, args = at*100,bounds=bnds).x

In [7]:
print('the optimal parameter for the model is a0:',w_n[0],'a1:',w_n[1],'b1:',w_n[2])

the optimal parameter for the model is a0: 0.04164982597470052 a1: 0.20772808669001702 b1: 0.7553679952874226


In [8]:
# compare the answer
from arch import arch_model

am = arch_model(at)
res = am.fit(update_freq=5)
print(res.summary())

Iteration:      5,   Func. Count:     64,   Neg. LLF: 198124657714.82193
Iteration:     10,   Func. Count:    135,   Neg. LLF: 2763047426417675.0
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4045.0169760938606
            Iterations: 15
            Function evaluations: 149
            Gradient evaluations: 11
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                      y   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                4045.02
Distribution:                  Normal   AIC:                          -8082.03
Method:            Maximum Likelihood   BIC:                          -8058.74
                                        No. Observations:                 2496
Date:                Wed, Dec 14 2022   Df Residuals:                     2495
Time:     

/opt/anaconda3/lib/python3.9/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001196. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


## b parameter for GEM ditribution 

In [9]:
w0_gem = [0.1,0.05,0.92,0.3]
bnds = ((0.001,None), (0.001,None),(0.001,None),(0.001,None))
w_gem = opt.minimize(ll_garch11, w0_gem, args = (at*100,'GED'),method='SLSQP',bounds=bnds).x

/var/folders/d0/6cymk_pn0yl4bjbhkblw54bh0000gn/T/ipykernel_62021/3288035068.py:9: RuntimeWarning: overflow encountered in double_scalars
  sig[i] = a0 + a1*at[i-1]**2 + b* sig[i-1]
/var/folders/d0/6cymk_pn0yl4bjbhkblw54bh0000gn/T/ipykernel_62021/3840848076.py:18: RuntimeWarning: invalid value encountered in log
  L -= sum(np.log(sig))/2
/var/folders/d0/6cymk_pn0yl4bjbhkblw54bh0000gn/T/ipykernel_62021/3840848076.py:19: RuntimeWarning: invalid value encountered in power
  L -= 1/2* sum((at**2/(sig* (lbd**2)))**(v/2))
/var/folders/d0/6cymk_pn0yl4bjbhkblw54bh0000gn/T/ipykernel_62021/3840848076.py:16: RuntimeWarning: invalid value encountered in double_scalars
  lbd = np.sqrt((2**(-2*iv)*gamma(iv)/gamma(3*iv)))


In [10]:
print('the optimal parameter for the model is a0:',w_gem[0],'a1:',w_gem[1],'b1:',w_gem[2],'v:',w_gem[3])

the optimal parameter for the model is a0: 0.03287438072148317 a1: 0.20047726735338767 b1: 0.7723167192044649 v: 1.2969297939132278


## c TGARCH(1,1) model

In [11]:
def sig_tgarch11(a0,a1,b,y,at):
    # at from a0 to at , length t+1
    n = len(at)
    sig = np.zeros(n)
    for i in range(len(at)):
        if i == 0 :
            sig[i] = a0/(1-a1-b)
        else:
            if at[i-1]< 0:
                sig[i] = a0 + (a1+y)*at[i-1]**2 + b* sig[i-1]
            else:
                sig[i] = a0 + a1*at[i-1]**2 + b* sig[i-1]
    return sig
# the last sig doesnt match any output

In [12]:
def ll_tgarch11(para, at):
    a0 = para[0]
    a1= para[1]
    b = para[2]
    y = para[3]

    sig = sig_tgarch11(a0,a1,b,y,at)
    n = len(at)
    L = 0
    L += (-n/2)*np.log(2*np.pi)
    L += np.sum(-np.log(sig)-at**2/sig)/2
    return -L

In [13]:
w0_tg  = [0.1,0.05,0.92,0.01]
bnds = ((0.001,None), (0.001,None),(0.001,None),(0.001,None))
w_tg = opt.minimize(ll_tgarch11, w0_tg, args = at*100, method='SLSQP',bounds=bnds).x

/var/folders/d0/6cymk_pn0yl4bjbhkblw54bh0000gn/T/ipykernel_62021/598726705.py:11: RuntimeWarning: invalid value encountered in log
  L += np.sum(-np.log(sig)-at**2/sig)/2


In [14]:
print('the optimal parameter for the model is a0:',w_tg[0],'a1:',w_tg[1],'b1:',w_tg[2],'y1:',w_tg[3])

the optimal parameter for the model is a0: 0.03790114647945401 a1: 0.07536032453742512 b1: 0.7644180538381198 y1: 0.2704068684851337


## d model comparion compare mle of bc and using Schwartz Bayesian Criteria to compare likelihood functions when the number of parameters is different.

In [15]:
logLa = -ll_garch11(w_n, at*100, distr='N')
logLb = -ll_garch11(w_gem, at*100, distr='GED')
logLc = -ll_tgarch11(w_tg, at*100)
print('mle for model in c is :',logLc,'mle for model in b is :',logLb)
print('based on the loglikelihood here, using model b has larger logL and considered as better than using c')

mle for model in c is : -3052.394065737338 mle for model in b is : -3022.49754178248
based on the loglikelihood here, using model b has larger logL and considered as better than using c


In [16]:
def SBC(mle,p,t):
    return -2*mle + p*np.log(t)
print('the result from SB criteria for model in a is :'
      ,SBC(logLa,3,len(at)),' for model in b is :',SBC(logLb,4,len(at)))
print('based on the Schwartz Bayesian Criteria here, using model b has smaller SBC and considered as better than using a')
print('overall model b is the best model among three.')

the result from SB criteria for model in a is : 6210.764485636474  for model in b is : 6076.284862482917
based on the Schwartz Bayesian Criteria here, using model b has smaller SBC and considered as better than using a
overall model b is the best model among three.


## 2 These options expire on Friday, December 9, 2022, which is 7 trading days from Wednesday, November 30. Assume that the annual risk-free rate, rf , is 4.71% and there are no dividends.

In [17]:
T = 7 / 252
rf = .0471
S, Kc, Kp = 407.68, 416, 400
N = 1000 # num of simulation 
n = 100 # num of interval


In [18]:
def gen_path(para,n):
    sig = np.zeros(n)
    a = np.zeros(n)
    sig[0] = para[0]/(1-para[1]-para[2]) #from the pic
    rand = np.random.standard_normal(n)
    a[0]  = np.sqrt(sig[0]) * rand[0]
    for i in range(1,n):
        if a[i-1]< 0:
            new = para[0] + (para[1]+ para[3])*a[i-1]**2 + para[2]* sig[i-1]
        else:
            new = para[0] + para[1]*a[i-1]**2 + para[2]* sig[i-1]
        sig[i] = new
        a[i] = np.sqrt(new)*rand[i]
    return sig/10 , a 
# need to adjust sigma ,since its from 100* return 
       

In [19]:
def MC_call(S,K,T,r,N,n,para):
    dt = T/n
    ret = 0
    sig_ = np.zeros([N,n])
    a_ = np.zeros([N,n])
    rand = np.random.normal(size=(N,n))
    for i in range(N):
        sig, a = gen_path(para,n)
        sig_[i] = sig
        a_[i] = a
    #sig_ = np.ones([N,n])*0.2
    S = S * np.exp(np.sum((r - 0.5 * sig_ ** 2) * dt + sig_* np.sqrt(dt)*rand , axis=1))
    C0 = np.exp(-r * T) * sum(np.maximum(S - K, 0)) / N
    return C0

def MC_put(S,K,T,r,N,n,para):
    dt = T/n
    ret = 0
    sig_ = np.zeros([N,n])
    a_ = np.zeros([N,n])
    rand = np.random.normal(size=(N,n))
    for i in range(N):
        sig, a = gen_path(para,n)
        sig_[i] = sig
        a_[i] = a
    S = S * np.exp(np.sum((r - 0.5 * sig_ ** 2) * dt + sig_* np.sqrt(dt)*rand , axis=1))
    C0 = np.exp(-r * T) * sum(np.maximum(K - S, 0)) / N
    return C0

In [20]:
np.random.seed(123)
call = MC_call(S,Kc,T,rf,10000,n,w_tg)
put = MC_put(S,Kp,T,rf,10000,n,w_tg)
print('call price by GARCH model is ',call)
print('put price by GARCH model is ', put)

call price by GARCH model is  1.8847576537460267
put price by GARCH model is  1.6603231505717297


In [21]:
from scipy.stats import norm

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T)* norm.cdf(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

In [22]:
from scipy.optimize import fsolve
def BS_IMVC(vol):
    return BS_CALL(S, Kc, T, rf, vol)-call
def BS_IMVP(vol):
    return BS_PUT(S, Kp, T, rf, vol)-put

call_IMV = fsolve(BS_IMVC,0.02)
put_IMV = fsolve(BS_IMVP,0.02)

In [23]:
print('The corresponding Implied Volatility for the Call price is ',call_IMV)
print('The corresponding Implied Volatility for the Put price is ',put_IMV)

The corresponding Implied Volatility for the Call price is  [0.17558805]
The corresponding Implied Volatility for the Put price is  [0.17356378]


## b TARCH Model and Skew of the options

In [24]:
print('Spread in the implied volatility is ',(call_IMV-put_IMV)*100)

Spread in the implied volatility is  [0.20242715]


The spread (skew) obtained from market data is larger than our model prediction and majority of past index, which suggest the market is uncertain about the call option value and lead to a increase in the volatility. With a believe the value will revert to a lower value I will long the put option can short the call option.

## 3 stochastic volatility model

In [25]:
df3 = pd.read_csv('SV_hw3.csv').iloc[:,1:].values.ravel()

In [83]:
# para = a, b, r
def e_s(i,a,b,r):
    return np.exp(i*a+(i*b)**2/2)

def e_e(i):
    return gamma((i+1)/2)*2**(i/2)/np.sqrt(np.pi)

def e_ss(i,a,b,r):
    return np.exp(2*a+b**2*(1+r**i))

def e_s2s2(i,a,b,r):
    return np.exp(4*a+4*b**2*(1+r**i))

def g1(para,arr):
    T = len(arr)
    l = sum(np.abs(arr))/T
    r = e_s(1,para[0],para[1],para[2])*e_e(1)
    return l-r

def g2(para,arr):
    T = len(arr)
    l = sum(arr**2)/T
    r = e_s(2,para[0],para[1],para[2])*e_e(2)
    return l-r

def g3(para,arr):
    T = len(arr)
    l = sum(np.abs(arr[1:])*np.abs(arr[:-1]))/(T-1)
    r = e_ss(1,para[0],para[1],para[2])*e_e(1)**2
    return l-r

def g4(para,arr):
    T = len(arr)
    l = sum(np.abs(arr[2:])*np.abs(arr[:-2]))/(T-2)
    r = e_ss(2,para[0],para[1],para[2])*e_e(1)**2
    return l-r

def g5(para,arr):
    T = len(arr)
    l = sum((arr[1:]*arr[:-1])**2)/(T-1)
    r = e_s2s2(1,para[0],para[1],para[2])*e_e(2)**2
    return l-r

def g_list(para,arr):
    g = np.array([[g1(para,arr),g2(para,arr),g3(para,arr),g4(para,arr),g5(para,arr)]])
    return g

def W(para,arr):
    g_ = g(para,arr)
    S = g_.T @ g_* len(arr)
    return np.linalg.inv(S)

In [84]:
def optmizer(para,arr,W):
    g_ = g(para,arr)
    return (g_@ W @g_.T)[0][0]

In [115]:
para0 = [0.1,0.1,0.1]
g0 = g_list(para0,df3)
fmean = np.mean([g_list(para0,df3[:i]) for i in range(4,len(df3))],axis=0)
S0 = len(df3-4)*(fmean.T@fmean)
W0 = np.linalg.pinv(S0) 
bnds = ((0.0001,None), (0.0001,None),(0.0001,None))
p_opt = opt.minimize(optmizer, para0, args = (df3,W0),bounds=bnds).x

In [116]:

fmean = np.mean([g_list(p_opt,df3[:i]) for i in range(4,len(df3))],axis=0)
S0 = len(df3-4)*(fmean.T@fmean)
W0 = np.linalg.pinv(S0) 
bnds = ((0.0001,None), (0.0001,None),(0.0001,None))
p_final = opt.minimize(optmizer, para0, args = (df3,W0),bounds=bnds).x

In [118]:
print('the optimal parameter for the model is alpha:',p_final[0],'beta:',p_final[1],'rho:',p_final[2])

the optimal parameter for the model is alpha: 0.0001 beta: 0.0028915480437934626 rho: 0.09894644703861646


In [120]:
print('the parameter for alpha is not accurate here,there are some problems in the optimization')

the parameter for alpha is not accurate here,there are some problems in the optimization
